## Layers in Graph Convolutional Network

In this notebook we will be seeing Graph ConvolutionalNetwork(GCN). GCN are based on structures of a graph, i.e, nodes, edges and weighted edges. But we will be only discussing non-weighted and non-message passing. We will be developing layers using Pytorch. We will input input and output for each layer. We initialize weight and bias as Parameters since in Activation function $y=A(Wx + b)$ W and b are Weight and biases with x as input and y as output. If Bias is given then just declare it as Parameter else you will have to [register_parameter](https://pytorch.org/docs/stable/nn.htmlhighlight=register_parameter#torch.nn.Module.register_parameter).

Then we reset the paremeters by standardizing it in range (-stdv,stdv) where stdv is one by square root of number of output features. Here weights.size() outputs the shape of tensor weights and weights.size(1) outputs the shape tuple's dimension 1. Till here our weights and biases are filled and we have number of output and input features. Now we will be forward propagating. 

For that we will need the input matrix and adjacent matrix. Here adjacent matrix is a sparse matrix which contains 1's where connection between nodes and 0 when there is no connection as the number of nodes increases this sparsity of matrix increases. So, storing mostly zeros is an inefficient way for graphs. For this purpose we have a scipy library for sparse function and here we use torch library for sparse multiplicaton.

Sparse matrix is stored in diferent format, i.e, only ones location are stored and rest auomatically are zero. This not just saves storage but also saves search complexity.

In Forward Propagation we input the graph matrix or output of previous layer and multiply it by weights and add bias to it so as to form the $Wx+ b$ form but here is a catch. But unlike convoltional neural network which does not considers values farther,i.e, considers or apply function only on what is inside filter our GCN takes all the nodes connected to it in using adjacent matrix. So we do a [sparse matrix multiplication](https://pytorch.org/docs/stable/sparse.html?highlight=spmm#torch.sparse.mm) on our output of $Wx$ matrix. Now we add our bias to it to get our good old form $A(Wx)+b$.

In [1]:
import math
import torch
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module

In [4]:
class GraphConvolutionalNetwork(Module):
    
    def __init__(self, in_features, out_features, bias=True):
        super(GraphConvolutionalNetwork, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weights = Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.bias = register_parameter('bias', None)
        self.reset_parameters()
        
    def reset_parameters(self):
        stdv = 1.0/math.sqrt(self.weights.size(1))
        self.weights.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)
            
    def forward(self, input_, adj):
        support = torch.mm(input_, self.weights)
        output = torch.sparse.mm(adj, support)
        if self.bias is not None:
            output+= self.bias
            return output
        else:
            return output
        
    def __repr__(self):
        print(self.__class__.__name__+"(" \
              + str(self.in_features)+"->" \
              + str(self.out_features))+")"

In [2]:
weights = Parameter(torch.FloatTensor(2, 3))

In [5]:
weights.size()

torch.Size([2, 3])